In [1]:
import lancedb
import pyarrow as pa
import torch
from PIL import Image
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from model import Model
import os
from collections import defaultdict
from tqdm import tqdm
import random
import hashlib

In [2]:
my_model = Model("lancedb")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
C:\Users\pc\.cache\huggingface\modules\transformers_modules\nomic-ai\nomic-bert-2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects wi

In [7]:
# single image use_replace = False
true_label = []
preds_top5 = []
val_dir = './data_train_test/val_dir'

for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    for img_name in tqdm(os.listdir(cls_dir), desc=f"{cls} process"):
        img_path = os.path.join(cls_dir, img_name)


        top5 = my_model.find_query_by_picture(img_path, num_data = 1000, use_replace = False, top_k = 5)
        names5 = [name for name, _, _ in top5]

        true_label.append(cls)
        preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall Recall@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Recall@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

Lichen planus process: 100%|██████████| 206/206 [00:25<00:00,  8.07it/s]
Lipoid proteinosis process: 0it [00:00, ?it/s]
Melanoma process: 100%|██████████| 539/539 [01:11<00:00,  7.54it/s]
Melkersson-rosenthal syndrome process: 0it [00:00, ?it/s]
Poison Ivy process: 100%|██████████| 7/7 [00:00<00:00,  7.67it/s]
Pseudoxanthoma elasticum process: 0it [00:00, ?it/s]
Vasculitis process: 100%|██████████| 8/8 [00:01<00:00,  7.55it/s]
Warts Molluscum and other Viral Infections process: 100%|██████████| 240/240 [00:30<00:00,  7.92it/s]

Overall Recall@5: 7611/7697 = 98.883%

Per-class Recall@5:
Acanthosis nigricans (   3 samples): 100.000%
Acrodermatitis enteropathica (   2 samples): 100.000%
Amyloidosis          (   2 samples): 100.000%
Basal Cell Carcinoma ( 554 samples): 100.000%
Birt-hogg-dube syndrome (   3 samples): 100.000%
Chickenpox           (  23 samples): 100.000%
Cowden syndrome      (   3 samples): 100.000%
Measles              (  26 samples): 100.000%
Monkeypox            (  16 samples): 100.000%
Neurofibromatosis    (   3 samples): 100.000%
Not a disease        ( 184 samples): 100.000%
Pellagra             (   1 samples): 100.000%
Tuberous sclerosis   (   3 samples): 100.000%
Urticaria Hives      (   3 samples): 100.000%
Vasculitis           (   8 samples): 100.000%
Melanocytic Nevi     ( 877 samples): 99.886%
Melanoma             ( 539 samples): 99.814%
Disease by infectious agent ( 749 samples): 99.599%
Benign               (1600 samples): 99.562%
Eczema               ( 509 samples): 99.411%
Malign  

In [3]:
# single image use_replace = True
true_label = []
preds_top5 = []
val_dir = './data_train_test/val_dir'

for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    for img_name in tqdm(os.listdir(cls_dir), desc=f"{cls} process"):
        img_path = os.path.join(cls_dir, img_name)


        top5 = my_model.find_query_by_picture(img_path, num_data = 1000, use_replace = True, top_k = 5)
        names5 = [name for name, _, _ in top5]

        true_label.append(cls)
        preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall Recall@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Recall@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

Acanthosis nigricans process:   0%|          | 0/3 [00:00<?, ?it/s]

Lichen planus process: 100%|██████████| 206/206 [01:15<00:00,  2.72it/s]
Lipoid proteinosis process: 0it [00:00, ?it/s]
Melanoma process: 100%|██████████| 539/539 [02:46<00:00,  3.25it/s]
Melkersson-rosenthal syndrome process: 0it [00:00, ?it/s]
Poison Ivy process: 100%|██████████| 7/7 [00:02<00:00,  3.32it/s]
Pseudoxanthoma elasticum process: 0it [00:00, ?it/s]
Vasculitis process: 100%|██████████| 8/8 [00:02<00:00,  3.14it/s]
Warts Molluscum and other Viral Infections process: 100%|██████████| 240/240 [01:17<00:00,  3.10it/s]

Overall Recall@5: 7610/7697 = 98.870%

Per-class Recall@5:
Acanthosis nigricans (   3 samples): 100.000%
Acrodermatitis enteropathica (   2 samples): 100.000%
Amyloidosis          (   2 samples): 100.000%
Basal Cell Carcinoma ( 554 samples): 100.000%
Birt-hogg-dube syndrome (   3 samples): 100.000%
Chickenpox           (  23 samples): 100.000%
Cowden syndrome      (   3 samples): 100.000%
Measles              (  26 samples): 100.000%
Monkeypox            (  16 samples): 100.000%
Neurofibromatosis    (   3 samples): 100.000%
Not a disease        ( 184 samples): 100.000%
Pellagra             (   1 samples): 100.000%
Urticaria Hives      (   3 samples): 100.000%
Vasculitis           (   8 samples): 100.000%
Melanocytic Nevi     ( 877 samples): 99.886%
Melanoma             ( 539 samples): 99.814%
Benign               (1600 samples): 99.562%
Disease by infectious agent ( 749 samples): 99.466%
Eczema               ( 509 samples): 99.411%
Malign               ( 848 samples): 99.410%
Acne     

In [3]:
# multi true image use_replace = False

val_dir = './data_train_test/val_dir'
random.seed(1234)
list_image = []
list_disease = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    all_images = [img for img in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, img))]

    num_rounds = max(len(all_images) // 4, 1)
    for i in range(num_rounds):
        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        num_image_samples = rng.randint(3, 5)

        sampled_images = rng.sample(all_images, min(num_image_samples, len(all_images)))
        if sampled_images == []:
            continue
        if sampled_images in list_image:
            print("random trùng nhau", sampled_images)
        list_image.append([os.path.join(cls_dir, img_name) for img_name in sampled_images])
        list_disease.append(cls)


true_label = []
preds_top5 = []
for image, disease in tqdm(zip(list_image, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_pictures(image_paths = image, num_data = 10000, top_k = 5, use_replace = False, metric = "cosine")
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall Recall@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Recall@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

process_data: 100%|██████████| 1919/1919 [31:45<00:00,  1.01it/s]

Overall Recall@5: 1918/1919 = 99.948%

Per-class Recall@5:
Acanthosis nigricans (   1 samples): 100.000%
Acne                 ( 124 samples): 100.000%
Acrodermatitis enteropathica (   1 samples): 100.000%
Actinic keratosis    (  30 samples): 100.000%
Addison's disease    (   1 samples): 100.000%
Amyloidosis          (   1 samples): 100.000%
Atopic dermatitis    (  39 samples): 100.000%
Basal Cell Carcinoma ( 138 samples): 100.000%
Benign               ( 400 samples): 100.000%
Birt-hogg-dube syndrome (   1 samples): 100.000%
Chickenpox           (   5 samples): 100.000%
Cowden syndrome      (   1 samples): 100.000%
Dermatofibroma       (  16 samples): 100.000%
Disease by infectious agent ( 187 samples): 100.000%
Eczema               ( 127 samples): 100.000%
Exanthem             (   1 samples): 100.000%
Herpes               (   5 samples): 100.000%
Lichen planus        (  51 samples): 100.000%
Malign               ( 212 samples): 100.000%
Measles              (   6 samples): 100.000%
Mel

In [4]:
# multi true image use_replace = True

val_dir = './data_train_test/val_dir'
random.seed(1234)
list_image = []
list_disease = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    all_images = [img for img in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, img))]

    num_rounds = max(len(all_images) // 4, 1)
    for i in range(num_rounds):
        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        num_image_samples = rng.randint(3, 5)

        sampled_images = rng.sample(all_images, min(num_image_samples, len(all_images)))
        if sampled_images == []:
            continue
        if sampled_images in list_image:
            print("random trùng nhau", sampled_images)
        list_image.append([os.path.join(cls_dir, img_name) for img_name in sampled_images])
        list_disease.append(cls)


true_label = []
preds_top5 = []
for image, disease in tqdm(zip(list_image, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_pictures(image_paths = image, num_data = 10000, top_k = 5, use_replace = True, metric = "cosine")
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall Recall@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Recall@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

process_data: 100%|██████████| 1919/1919 [1:06:07<00:00,  2.07s/it]

Overall Recall@5: 1918/1919 = 99.948%

Per-class Recall@5:
Acanthosis nigricans (   1 samples): 100.000%
Acne                 ( 124 samples): 100.000%
Acrodermatitis enteropathica (   1 samples): 100.000%
Actinic keratosis    (  30 samples): 100.000%
Addison's disease    (   1 samples): 100.000%
Amyloidosis          (   1 samples): 100.000%
Atopic dermatitis    (  39 samples): 100.000%
Basal Cell Carcinoma ( 138 samples): 100.000%
Benign               ( 400 samples): 100.000%
Birt-hogg-dube syndrome (   1 samples): 100.000%
Chickenpox           (   5 samples): 100.000%
Cowden syndrome      (   1 samples): 100.000%
Dermatofibroma       (  16 samples): 100.000%
Disease by infectious agent ( 187 samples): 100.000%
Eczema               ( 127 samples): 100.000%
Exanthem             (   1 samples): 100.000%
Herpes               (   5 samples): 100.000%
Lichen planus        (  51 samples): 100.000%
Malign               ( 212 samples): 100.000%
Measles              (   6 samples): 100.000%
Mel

In [13]:
# multi with false image use_replace = False

val_dir = './data_train_test/val_dir'
random.seed(1234)
list_image = []
list_disease = []

all_classes = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    if os.path.isdir(cls_dir):
        imgs = [f for f in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, f))]
        all_classes.append((cls, cls_dir, imgs))

print(all_classes)

for cls, cls_dir, images in all_classes:

    num_rounds = max(len(images) // 4, 1)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        total_N = rng.randint(3, 5)

        other_choices = [
            (o_cls, o_dir, o_imgs)
            for (o_cls, o_dir, o_imgs) in all_classes
            if o_cls != cls and len(o_imgs) > 0
        ]
        if not other_choices:
            sampled_main = rng.sample(images, min(total_N, len(images)))
            batch = sampled_main
        else:
            o_cls, o_dir, o_imgs = rng.choice(other_choices)
            one_other = rng.choice(o_imgs)
            k_main = min(total_N - 1, len(images))
            sampled_main = rng.sample(images, k_main)
            batch = [os.path.join(o_dir, one_other)] + [os.path.join(cls_dir, img) for img in sampled_main]

        list_image.append(batch)
        list_disease.append(cls)

print(len(list_image), list_image)
print(len(list_disease), list_disease)

true_label = []
preds_top5 = []
for image, disease in tqdm(zip(list_image, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_pictures(image_paths = image, num_data = 10000, top_k = 5, use_replace = False, metric = "cosine")
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall Recall@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Recall@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

[('Acanthosis nigricans', './data_train_test/val_dir\\Acanthosis nigricans', ['Acanthosis-nigricans-and-skin-tags-over-A-neck-B-left-axillae-before-surgery.jpg', 'images (10).jpg', 'images (3).jpg']), ('Acne', './data_train_test/val_dir\\Acne', ['01F3MMV6NBB013AEV2SH04SRQ9_jpeg.rf.a88bc38371062a738c5591586ce56307.jpg', '01F3MMVC4SSD2BWWP115BMSB3Q_jpeg.rf.5b15bc9ffb18a63f7eea967c595d75cd.jpg', '01F3MMVJ16XHRQ3TM1JQANVQXV_jpeg.rf.b8f7668dfe0a20717f78b8dbfafe4d4f.jpg', '01F3MMVNR3XB2A86H1Y87XW2SE_jpeg.rf.9390b3b1096a4cca77c2dbfd3f2d967f.jpg', '01F3MMVZF3PER8R7TWBDDT91S3_jpeg.rf.c81abac87e892ed07347e5c5835c2f35.jpg', '01F3MMW4EVZS8XK1FV5PMP2NXP_jpeg.rf.2e34cc3e72cd10ff474651883d1c7cff.jpg', '01F3MMWA0ZXZAHHVH6VFY7VSEH_jpeg.rf.fa98442c41ed55039e72b1905cd09f28.jpg', '01F3MMWK2WKFGMNCB4VC9D4CHD_jpeg.rf.78e6794cd94ff2ab6e3313a0dcca7ddc.jpg', '01F3MMWXCRJZGN9E33KZPXT82D_jpeg.rf.bde78cc2e1f2f36f393c182bc0a4aff7.jpg', '01F3MMWYSD0H8X9217EFP9VCYN_jpeg.rf.a1a4a95ba50eec2cddbb16b0f48549f1.jpg', '01F

process_data: 100%|██████████| 1922/1922 [34:15<00:00,  1.07s/it] 

Overall Recall@5: 1767/1922 = 91.935%

Per-class Recall@5:
Amyloidosis          (   1 samples): 100.000%
Benign               ( 400 samples): 100.000%
Birt-hogg-dube syndrome (   1 samples): 100.000%
Chickenpox           (   5 samples): 100.000%
Dermatofibroma       (  16 samples): 100.000%
Exanthem             (   1 samples): 100.000%
Malign               ( 212 samples): 100.000%
Pellagra             (   1 samples): 100.000%
Poison Ivy           (   1 samples): 100.000%
Urticaria Hives      (   1 samples): 100.000%
Vascular Tumors      (  23 samples): 100.000%
Atopic dermatitis    (  39 samples): 97.436%
Eczema               ( 127 samples): 94.488%
Disease by infectious agent ( 187 samples): 94.118%
Melanoma             ( 134 samples): 94.030%
Actinic keratosis    (  30 samples): 93.333%
Basal Cell Carcinoma ( 138 samples): 90.580%
Squamous cell carcinoma (  21 samples): 90.476%
Warts Molluscum and other Viral Infections (  60 samples): 90.000%
Melanocytic Nevi     ( 219 samples): 88.

In [14]:
# multi with false image use_replace = True

val_dir = './data_train_test/val_dir'
random.seed(1234)
list_image = []
list_disease = []

all_classes = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    if os.path.isdir(cls_dir):
        imgs = [f for f in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, f))]
        all_classes.append((cls, cls_dir, imgs))

print(all_classes)

for cls, cls_dir, images in all_classes:

    num_rounds = max(len(images) // 4, 1)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        total_N = rng.randint(3, 5)

        other_choices = [
            (o_cls, o_dir, o_imgs)
            for (o_cls, o_dir, o_imgs) in all_classes
            if o_cls != cls and len(o_imgs) > 0
        ]
        if not other_choices:
            sampled_main = rng.sample(images, min(total_N, len(images)))
            batch = sampled_main
        else:
            o_cls, o_dir, o_imgs = rng.choice(other_choices)
            one_other = rng.choice(o_imgs)
            k_main = min(total_N - 1, len(images))
            sampled_main = rng.sample(images, k_main)
            batch = [os.path.join(o_dir, one_other)] + [os.path.join(cls_dir, img) for img in sampled_main]

        list_image.append(batch)
        list_disease.append(cls)

print(len(list_image), list_image)
print(len(list_disease), list_disease)

true_label = []
preds_top5 = []
for image, disease in tqdm(zip(list_image, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_pictures(image_paths = image, num_data = 10000, top_k = 5, use_replace = True, metric = "cosine")
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall Recall@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Recall@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

[('Acanthosis nigricans', './data_train_test/val_dir\\Acanthosis nigricans', ['Acanthosis-nigricans-and-skin-tags-over-A-neck-B-left-axillae-before-surgery.jpg', 'images (10).jpg', 'images (3).jpg']), ('Acne', './data_train_test/val_dir\\Acne', ['01F3MMV6NBB013AEV2SH04SRQ9_jpeg.rf.a88bc38371062a738c5591586ce56307.jpg', '01F3MMVC4SSD2BWWP115BMSB3Q_jpeg.rf.5b15bc9ffb18a63f7eea967c595d75cd.jpg', '01F3MMVJ16XHRQ3TM1JQANVQXV_jpeg.rf.b8f7668dfe0a20717f78b8dbfafe4d4f.jpg', '01F3MMVNR3XB2A86H1Y87XW2SE_jpeg.rf.9390b3b1096a4cca77c2dbfd3f2d967f.jpg', '01F3MMVZF3PER8R7TWBDDT91S3_jpeg.rf.c81abac87e892ed07347e5c5835c2f35.jpg', '01F3MMW4EVZS8XK1FV5PMP2NXP_jpeg.rf.2e34cc3e72cd10ff474651883d1c7cff.jpg', '01F3MMWA0ZXZAHHVH6VFY7VSEH_jpeg.rf.fa98442c41ed55039e72b1905cd09f28.jpg', '01F3MMWK2WKFGMNCB4VC9D4CHD_jpeg.rf.78e6794cd94ff2ab6e3313a0dcca7ddc.jpg', '01F3MMWXCRJZGN9E33KZPXT82D_jpeg.rf.bde78cc2e1f2f36f393c182bc0a4aff7.jpg', '01F3MMWYSD0H8X9217EFP9VCYN_jpeg.rf.a1a4a95ba50eec2cddbb16b0f48549f1.jpg', '01F

process_data: 100%|██████████| 1922/1922 [1:05:37<00:00,  2.05s/it]

Overall Recall@5: 1762/1922 = 91.675%

Per-class Recall@5:
Amyloidosis          (   1 samples): 100.000%
Benign               ( 400 samples): 100.000%
Birt-hogg-dube syndrome (   1 samples): 100.000%
Chickenpox           (   5 samples): 100.000%
Dermatofibroma       (  16 samples): 100.000%
Exanthem             (   1 samples): 100.000%
Malign               ( 212 samples): 100.000%
Measles              (   6 samples): 100.000%
Pellagra             (   1 samples): 100.000%
Poison Ivy           (   1 samples): 100.000%
Urticaria Hives      (   1 samples): 100.000%
Vascular Tumors      (  23 samples): 100.000%
Atopic dermatitis    (  39 samples): 97.436%
Eczema               ( 127 samples): 94.488%
Disease by infectious agent ( 187 samples): 93.583%
Actinic keratosis    (  30 samples): 93.333%
Melanoma             ( 134 samples): 93.284%
Warts Molluscum and other Viral Infections (  60 samples): 91.667%
Basal Cell Carcinoma ( 138 samples): 90.580%
Squamous cell carcinoma (  21 samples): 90

In [16]:
# single out of sample use_replace = False
true_label = []
preds_top5 = []
val_dir = './out_of_sample'
i = 0
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    for img_name in tqdm(os.listdir(cls_dir), desc=f"{cls} process"):
        img_path = os.path.join(cls_dir, img_name)


        top5 = my_model.find_query_by_picture(img_path, num_data = 10000, top_k = 5, use_replace = False, metric = "cosine")
        names5 = [name for name, _, _ in top5]

        true_label.append(cls) 
        preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

Vasculitis process: 100%|██████████| 5/5 [00:01<00:00,  3.76it/s]
Warts Molluscum and other Viral Infections process: 100%|██████████| 5/5 [00:01<00:00,  4.28it/s]

Overall accuracy@5: 120/200 = 60.000%

Per-class Accuracy@5:
Acanthosis nigricans (   5 samples): 100.000%
Acne                 (   5 samples): 100.000%
Atopic dermatitis    (   5 samples): 100.000%
Benign               (   5 samples): 100.000%
Disease by infectious agent (   5 samples): 100.000%
Eczema               (   5 samples): 100.000%
Lichen planus        (   5 samples): 100.000%
Melanoma             (   5 samples): 100.000%
Not a disease        (   5 samples): 100.000%
Sebaceous adenocarcinoma (   5 samples): 100.000%
Basal Cell Carcinoma (   5 samples): 80.000%
Chickenpox           (   5 samples): 80.000%
Malign               (   5 samples): 80.000%
Tinea Ringworm Candidiases (   5 samples): 80.000%
Urticaria Hives      (   5 samples): 80.000%
Warts Molluscum and other Viral Infections (   5 samples): 80.000%
Cowden syndrome      (   6 samples): 66.667%
Acrodermatitis enteropathica (   5 samples): 60.000%
Actinic keratosis    (   5 samples): 60.000%
Amyloidosis          (   5 

In [15]:
# single out of sample use_replace = True
true_label = []
preds_top5 = []
val_dir = './out_of_sample'
i = 0
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    for img_name in tqdm(os.listdir(cls_dir), desc=f"{cls} process"):
        img_path = os.path.join(cls_dir, img_name)


        top5 = my_model.find_query_by_picture(img_path, num_data = 10000, top_k = 5, use_replace = True, metric = "cosine")
        names5 = [name for name, _, _ in top5]

        true_label.append(cls) 
        preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

Vasculitis process: 100%|██████████| 5/5 [00:02<00:00,  1.77it/s]
Warts Molluscum and other Viral Infections process: 100%|██████████| 5/5 [00:02<00:00,  2.08it/s]

Overall accuracy@5: 120/200 = 60.000%

Per-class Accuracy@5:
Acanthosis nigricans (   5 samples): 100.000%
Acne                 (   5 samples): 100.000%
Atopic dermatitis    (   5 samples): 100.000%
Benign               (   5 samples): 100.000%
Disease by infectious agent (   5 samples): 100.000%
Eczema               (   5 samples): 100.000%
Lichen planus        (   5 samples): 100.000%
Melanoma             (   5 samples): 100.000%
Not a disease        (   5 samples): 100.000%
Sebaceous adenocarcinoma (   5 samples): 100.000%
Basal Cell Carcinoma (   5 samples): 80.000%
Chickenpox           (   5 samples): 80.000%
Malign               (   5 samples): 80.000%
Tinea Ringworm Candidiases (   5 samples): 80.000%
Urticaria Hives      (   5 samples): 80.000%
Warts Molluscum and other Viral Infections (   5 samples): 80.000%
Cowden syndrome      (   6 samples): 66.667%
Acrodermatitis enteropathica (   5 samples): 60.000%
Actinic keratosis    (   5 samples): 60.000%
Amyloidosis          (   5 

In [5]:
# multi true out of sample use_replace = False

val_dir = './out_of_sample'
random.seed(1234)
list_image = []
list_disease = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    all_images = [img for img in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, img))]

    num_rounds = max(len(all_images) // 2, 1)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        num_image_samples = rng.randint(3, 5)

        sampled_images = rng.sample(all_images, min(num_image_samples, len(all_images)))
        if sampled_images in list_image:
            print("random trùng nhau")
        list_image.append([os.path.join(cls_dir, img_name) for img_name in sampled_images])
        list_disease.append(cls)


true_label = []
preds_top5 = []
for image, disease in tqdm(zip(list_image, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_pictures(image_paths = image, num_data = 1000, top_k = 5, use_replace = False, metric = "cosine")
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall Recall@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Recall@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

process_data: 100%|██████████| 83/83 [00:43<00:00,  1.91it/s]

Overall Recall@5: 60/83 = 72.289%

Per-class Recall@5:
Acanthosis nigricans (   2 samples): 100.000%
Acne                 (   2 samples): 100.000%
Addison's disease    (   2 samples): 100.000%
Amyloidosis          (   2 samples): 100.000%
Atopic dermatitis    (   2 samples): 100.000%
Basal Cell Carcinoma (   2 samples): 100.000%
Benign               (   2 samples): 100.000%
Birt-hogg-dube syndrome (   2 samples): 100.000%
Chickenpox           (   2 samples): 100.000%
Cowden syndrome      (   3 samples): 100.000%
Disease by infectious agent (   2 samples): 100.000%
Eczema               (   2 samples): 100.000%
Herpes               (   2 samples): 100.000%
Lichen planus        (   2 samples): 100.000%
Malign               (   2 samples): 100.000%
Melanocytic Nevi     (   3 samples): 100.000%
Melanoma             (   2 samples): 100.000%
Neurofibromatosis    (   2 samples): 100.000%
Nevus                (   2 samples): 100.000%
Not a disease        (   2 samples): 100.000%
Pellagra       

In [6]:
# multi true out of sample use_replace = True

val_dir = './out_of_sample'
random.seed(1234)
list_image = []
list_disease = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    all_images = [img for img in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, img))]

    num_rounds = max(len(all_images) // 2, 1)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        num_image_samples = rng.randint(3, 5)

        sampled_images = rng.sample(all_images, min(num_image_samples, len(all_images)))
        if sampled_images in list_image:
            print("random trùng nhau")
        list_image.append([os.path.join(cls_dir, img_name) for img_name in sampled_images])
        list_disease.append(cls)


true_label = []
preds_top5 = []
for image, disease in tqdm(zip(list_image, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_pictures(image_paths = image, num_data = 1000, top_k = 5, use_replace = True, metric = "cosine")
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall Recall@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Recall@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

process_data: 100%|██████████| 83/83 [01:38<00:00,  1.19s/it]

Overall Recall@5: 60/83 = 72.289%

Per-class Recall@5:
Acanthosis nigricans (   2 samples): 100.000%
Acne                 (   2 samples): 100.000%
Addison's disease    (   2 samples): 100.000%
Amyloidosis          (   2 samples): 100.000%
Atopic dermatitis    (   2 samples): 100.000%
Basal Cell Carcinoma (   2 samples): 100.000%
Benign               (   2 samples): 100.000%
Birt-hogg-dube syndrome (   2 samples): 100.000%
Chickenpox           (   2 samples): 100.000%
Cowden syndrome      (   3 samples): 100.000%
Disease by infectious agent (   2 samples): 100.000%
Eczema               (   2 samples): 100.000%
Herpes               (   2 samples): 100.000%
Lichen planus        (   2 samples): 100.000%
Malign               (   2 samples): 100.000%
Melanocytic Nevi     (   3 samples): 100.000%
Melanoma             (   2 samples): 100.000%
Neurofibromatosis    (   2 samples): 100.000%
Nevus                (   2 samples): 100.000%
Not a disease        (   2 samples): 100.000%
Pellagra       

In [17]:
# multi with false out of sample use_replace = False

val_dir = './out_of_sample'
random.seed(1234)
list_image = []
list_disease = []

all_classes = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    if os.path.isdir(cls_dir):
        imgs = [f for f in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, f))]
        all_classes.append((cls, cls_dir, imgs))

print(all_classes)

for cls, cls_dir, images in all_classes:

    num_rounds = max(len(images) // 2, 1)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        total_N = rng.randint(3, 5)

        other_choices = [
            (o_cls, o_dir, o_imgs)
            for (o_cls, o_dir, o_imgs) in all_classes
            if o_cls != cls and len(o_imgs) > 0
        ]
        if not other_choices:
            sampled_main = rng.sample(images, min(total_N, len(images)))
            batch = sampled_main
        else:
            o_cls, o_dir, o_imgs = rng.choice(other_choices)
            one_other = rng.choice(o_imgs)
            k_main = min(total_N - 1, len(images))
            sampled_main = rng.sample(images, k_main)
            batch = [os.path.join(o_dir, one_other)] + [os.path.join(cls_dir, img) for img in sampled_main]

        list_image.append(batch)
        list_disease.append(cls)

print(len(list_image), list_image)
print(len(list_disease), list_disease)

true_label = []
preds_top5 = []
for image, disease in tqdm(zip(list_image, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_pictures(image_paths = image, num_data = 1000, top_k = 5, use_replace = False, metric = "cosine")
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall Recall@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Recall@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

[('Acanthosis nigricans', './out_of_sample\\Acanthosis nigricans', ['Acan_nig_axil_2615addf7111d83624b7a7a5f3466fb3.jpg', 'A_0917_acanthosis-nigricans_C5HE23.original.max-600x600.jpg', 'ch1509.fig1_.jpg', 'S_0318_Acanthosis_nigricans_C0373908.origina.max-600x600.jpg', 'Treatment-for-Acanthosis-Nigricans.png']), ('Acne', './out_of_sample\\Acne', ['2356.jpg', 'acne-face-2-18.jpg', 'acne.jpg', 'S_0917_acne_M1080444.original.max-600x600.jpg', 'unnamed.jpg']), ('Acrodermatitis enteropathica', './out_of_sample\\Acrodermatitis enteropathica', ['14283268-Acrodermatitis-enteropathica.jpg', 'A-child-with-classical-clinical-picture-of-acrodermatitis-enteropathica-at-the-time-of_Q320.jpg', 'acroderm1__WatermarkedWyJXYXRlcm1hcmtlZCJd.jpg', 'Acrodermatitis-enteropathica-photograph-credit-Albert-Yan-MD.png', 'Acrodermatitis-enteropathica.jpg']), ('Actinic keratosis', './out_of_sample\\Actinic keratosis', ['52-300x300.png', 'actinic-keratoses.jpg', 'Actinic-Keratosis-Solar-Keratosis.jpg', 'ak.jpg', 'h

process_data: 100%|██████████| 83/83 [00:46<00:00,  1.77it/s]

Overall Recall@5: 34/83 = 40.964%

Per-class Recall@5:
Acne                 (   2 samples): 100.000%
Actinic keratosis    (   2 samples): 100.000%
Atopic dermatitis    (   2 samples): 100.000%
Basal Cell Carcinoma (   2 samples): 100.000%
Benign               (   2 samples): 100.000%
Birt-hogg-dube syndrome (   2 samples): 100.000%
Chickenpox           (   2 samples): 100.000%
Eczema               (   2 samples): 100.000%
Lichen planus        (   2 samples): 100.000%
Malign               (   2 samples): 100.000%
Melanocytic Nevi     (   3 samples): 100.000%
Melanoma             (   2 samples): 100.000%
Warts Molluscum and other Viral Infections (   2 samples): 100.000%
Disease by infectious agent (   2 samples): 50.000%
Herpes               (   2 samples): 50.000%
Nevus                (   2 samples): 50.000%
Tinea Ringworm Candidiases (   2 samples): 50.000%
Vascular Tumors      (   2 samples): 50.000%
Cowden syndrome      (   3 samples): 33.333%
Psoriasis            (   3 samples): 33

In [18]:
# multi with false out of sample use_replace = True

val_dir = './out_of_sample'
random.seed(1234)
list_image = []
list_disease = []

all_classes = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    if os.path.isdir(cls_dir):
        imgs = [f for f in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, f))]
        all_classes.append((cls, cls_dir, imgs))

print(all_classes)

for cls, cls_dir, images in all_classes:

    num_rounds = max(len(images) // 2, 1)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        total_N = rng.randint(3, 5)

        other_choices = [
            (o_cls, o_dir, o_imgs)
            for (o_cls, o_dir, o_imgs) in all_classes
            if o_cls != cls and len(o_imgs) > 0
        ]
        if not other_choices:
            sampled_main = rng.sample(images, min(total_N, len(images)))
            batch = sampled_main
        else:
            o_cls, o_dir, o_imgs = rng.choice(other_choices)
            one_other = rng.choice(o_imgs)
            k_main = min(total_N - 1, len(images))
            sampled_main = rng.sample(images, k_main)
            batch = [os.path.join(o_dir, one_other)] + [os.path.join(cls_dir, img) for img in sampled_main]

        list_image.append(batch)
        list_disease.append(cls)

print(len(list_image), list_image)
print(len(list_disease), list_disease)

true_label = []
preds_top5 = []
for image, disease in tqdm(zip(list_image, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_pictures(image_paths = image, num_data = 1000, top_k = 5, use_replace = False, metric = "cosine")
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall Recall@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Recall@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

[('Acanthosis nigricans', './out_of_sample\\Acanthosis nigricans', ['Acan_nig_axil_2615addf7111d83624b7a7a5f3466fb3.jpg', 'A_0917_acanthosis-nigricans_C5HE23.original.max-600x600.jpg', 'ch1509.fig1_.jpg', 'S_0318_Acanthosis_nigricans_C0373908.origina.max-600x600.jpg', 'Treatment-for-Acanthosis-Nigricans.png']), ('Acne', './out_of_sample\\Acne', ['2356.jpg', 'acne-face-2-18.jpg', 'acne.jpg', 'S_0917_acne_M1080444.original.max-600x600.jpg', 'unnamed.jpg']), ('Acrodermatitis enteropathica', './out_of_sample\\Acrodermatitis enteropathica', ['14283268-Acrodermatitis-enteropathica.jpg', 'A-child-with-classical-clinical-picture-of-acrodermatitis-enteropathica-at-the-time-of_Q320.jpg', 'acroderm1__WatermarkedWyJXYXRlcm1hcmtlZCJd.jpg', 'Acrodermatitis-enteropathica-photograph-credit-Albert-Yan-MD.png', 'Acrodermatitis-enteropathica.jpg']), ('Actinic keratosis', './out_of_sample\\Actinic keratosis', ['52-300x300.png', 'actinic-keratoses.jpg', 'Actinic-Keratosis-Solar-Keratosis.jpg', 'ak.jpg', 'h

process_data: 100%|██████████| 83/83 [00:47<00:00,  1.76it/s]

Overall Recall@5: 34/83 = 40.964%

Per-class Recall@5:
Acne                 (   2 samples): 100.000%
Actinic keratosis    (   2 samples): 100.000%
Atopic dermatitis    (   2 samples): 100.000%
Basal Cell Carcinoma (   2 samples): 100.000%
Benign               (   2 samples): 100.000%
Birt-hogg-dube syndrome (   2 samples): 100.000%
Chickenpox           (   2 samples): 100.000%
Eczema               (   2 samples): 100.000%
Lichen planus        (   2 samples): 100.000%
Malign               (   2 samples): 100.000%
Melanocytic Nevi     (   3 samples): 100.000%
Melanoma             (   2 samples): 100.000%
Warts Molluscum and other Viral Infections (   2 samples): 100.000%
Disease by infectious agent (   2 samples): 50.000%
Herpes               (   2 samples): 50.000%
Nevus                (   2 samples): 50.000%
Tinea Ringworm Candidiases (   2 samples): 50.000%
Vascular Tumors      (   2 samples): 50.000%
Cowden syndrome      (   3 samples): 33.333%
Psoriasis            (   3 samples): 33

In [19]:
# single symptom
symptom_list = []
disease_list = []
num_symptom_samples = 1
with open("test_symptom.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        symptom_list.append(random.sample(obj['used_symptoms'], 1))
        disease_list.append(obj['original_name'])


true_label = []
preds_top5 = []

for symptom, disease in tqdm(zip(symptom_list, disease_list), total=len(symptom_list), desc="process_data"):

    top5 = my_model.find_query_by_symptoms(symptom, num_data = 10000, top_k = 5)
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)
total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

process_data: 100%|██████████| 10077/10077 [41:24<00:00,  4.06it/s]

Overall accuracy@5: 4990/10077 = 49.519%

Per-class Accuracy@5:
A53 diffuse large b-cell lymphoma (   1 samples): 100.000%
Aagenaes syndrome    (   1 samples): 100.000%
Aarskog syndrome     (   1 samples): 100.000%
Abcd syndrome        (   1 samples): 100.000%
Abdominal aortic aneurysm (   1 samples): 100.000%
Abdominal obesity-metabolic syndrome 1 (   1 samples): 100.000%
Abdominal obesity-metabolic syndrome 3 (   1 samples): 100.000%
Abdominal obesity-metabolic syndrome 4 (   1 samples): 100.000%
Abnormal pupillary function (   1 samples): 100.000%
Abnormal retinal correspondence (   1 samples): 100.000%
Absence epilepsy     (   1 samples): 100.000%
Absolute glaucoma    (   1 samples): 100.000%
Acalculous cholecystitis (   1 samples): 100.000%
Acantholytic acanthoma (   1 samples): 100.000%
Acanthoma            (   1 samples): 100.000%
Acanthosis nigricans (   1 samples): 100.000%
Acatalasia           (   1 samples): 100.000%
Accommodative spasm  (   1 samples): 100.000%
Achard syndr

In [20]:
# multi true symptom
symptom_list = []
disease_list = []

with open("test_symptom.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        symptom_list.append(obj['used_symptoms'])
        disease_list.append(obj['original_name'])

true_label = []
preds_top5 = []

for symptom, disease in tqdm(zip(symptom_list, disease_list), total=len(symptom_list), desc="process_data"):

    top5 = my_model.find_query_by_symptoms(symptom, num_data = 1000, top_k = 5)
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)
total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

process_data: 100%|██████████| 10077/10077 [1:30:18<00:00,  1.86it/s]

Overall accuracy@5: 8918/10077 = 88.499%

Per-class Accuracy@5:
A53 diffuse large b-cell lymphoma (   1 samples): 100.000%
Aagenaes syndrome    (   1 samples): 100.000%
Aarskog syndrome     (   1 samples): 100.000%
Abacavir allergy     (   1 samples): 100.000%
Abcd syndrome        (   1 samples): 100.000%
Abdominal aortic aneurysm (   1 samples): 100.000%
Abdominal obesity-metabolic syndrome (   1 samples): 100.000%
Abdominal obesity-metabolic syndrome 1 (   1 samples): 100.000%
Abdominal obesity-metabolic syndrome 3 (   1 samples): 100.000%
Abdominal obesity-metabolic syndrome 4 (   1 samples): 100.000%
Abducens nerve neoplasm (   1 samples): 100.000%
Abducens nerve palsy (   1 samples): 100.000%
Abetalipoproteinemia (   1 samples): 100.000%
Ablepharon macrostomia syndrome (   1 samples): 100.000%
Abnormality of glucagon secretion (   1 samples): 100.000%
Abnormal pupillary function (   1 samples): 100.000%
Abnormal retinal correspondence (   1 samples): 100.000%
Abnormal threshold of

In [21]:
# multi with false symptom
random.seed(1234)
symptom_list = []
disease_list = []

all_classes = []
with open("test_symptom.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        all_classes.append((obj['original_name'], obj['used_symptoms']))


for disease, symptoms in tqdm(all_classes, "process all_classes"):
    num_rounds = max(len(symptoms) // 4, 1)
    for i in range(num_rounds):

        key = f"{disease}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        total_N = rng.randint(2,4)

        other_choices = [
            (o_disease, o_symptoms)
            for (o_disease, o_symptoms) in all_classes
            if o_disease != disease and len(o_symptoms) > 0
        ]
        if not other_choices:
            sampled_main = rng.sample(symptoms, min(total_N, len(symptoms)))
            batch = sampled_main
        else:
            o_disease, o_symptoms = rng.choice(other_choices)
            one_other = rng.choice(o_symptoms)
            k_main = min(total_N - 1, len(symptoms))
            sampled_main = rng.sample(symptoms, k_main)
            batch = [one_other] + sampled_main

        disease_list.append(disease)
        symptom_list.append(batch)
        
print(len(disease_list))
print(len(symptom_list))

true_label = []
preds_top5 = []

for symptom, disease in tqdm(zip(symptom_list, disease_list), total=len(symptom_list), desc="process_data"):

    top5 = my_model.find_query_by_symptoms(symptom, num_data = 1000, top_k = 5)
    names5 = [name for name, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)
total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

process all_classes: 100%|██████████| 10077/10077 [02:18<00:00, 72.81it/s]


11541
11541


process_data: 100%|██████████| 11541/11541 [1:12:24<00:00,  2.66it/s]

Overall accuracy@5: 4763/11541 = 41.270%

Per-class Accuracy@5:
A53 diffuse large b-cell lymphoma (   1 samples): 100.000%
Aagenaes syndrome    (   1 samples): 100.000%
Aarskog syndrome     (   1 samples): 100.000%
Abcd syndrome        (   1 samples): 100.000%
Abdominal aortic aneurysm (   1 samples): 100.000%
Ablepharon macrostomia syndrome (   1 samples): 100.000%
Abnormal threshold of rods (   1 samples): 100.000%
Absence epilepsy     (   1 samples): 100.000%
Acanthocephaliasis   (   1 samples): 100.000%
Acantholytic acanthoma (   1 samples): 100.000%
Acanthosis nigricans (   1 samples): 100.000%
Acatalasia           (   1 samples): 100.000%
Achalasia            (   2 samples): 100.000%
Achalasia microcephaly syndrome (   2 samples): 100.000%
Achard syndrome      (   1 samples): 100.000%
Achondrogenesis      (   1 samples): 100.000%
Achondrogenesis type ia (   1 samples): 100.000%
Achondrogenesis type ib (   1 samples): 100.000%
Achondrogenesis type ii (   1 samples): 100.000%
Achon

In [22]:
# single out of sample + symptomp use_replace = False
true_label = []
preds_top5 = []
val_dir = './out_of_sample'
random.seed(1234)
list_image = []
list_disease = []
list_symptom = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    all_images = [img for img in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, img))]
    
    num_rounds = random.randint(2, 4)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        num_image_samples = 1
        num_symptom_samples = 1

        sampled_images = rng.sample(all_images, num_image_samples)
        if sampled_images == []:
            continue
        if sampled_images in list_image:
            print("random trùng nhau", sampled_images)
        list_image.append([os.path.join(cls_dir, img_name) for img_name in sampled_images])
        list_disease.append(cls)


for disease in list_disease:

    key = f"{disease}".encode()
    seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
    rng = random.Random(seed)

    if disease == "Not a disease":
        list_symptom.append([])
    with open("test_symptom.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            if obj['original_name'] != disease:
                continue
            sampled_symptom = rng.sample(obj['used_symptoms'], num_symptom_samples)
            list_symptom.append(sampled_symptom)
            break
print(len(list_disease), list_disease)
print(len(list_image), list_image)
print(len(list_symptom), list_symptom)


true_label = []
preds_top5 = []

for image, symptom, disease in tqdm(zip(list_image, list_symptom, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_picture_symptom(list_picture=image, list_symptom=symptom, num_data=1000, top_k=5, use_replace = False, metric="cosine")
    names5 = [name for name, _, _, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

111 ['Acanthosis nigricans', 'Acanthosis nigricans', 'Acanthosis nigricans', 'Acne', 'Acne', 'Acrodermatitis enteropathica', 'Acrodermatitis enteropathica', 'Actinic keratosis', 'Actinic keratosis', "Addison's disease", "Addison's disease", "Addison's disease", "Addison's disease", 'Amyloidosis', 'Amyloidosis', 'Atopic dermatitis', 'Atopic dermatitis', 'Atopic dermatitis', 'Atopic dermatitis', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Benign', 'Benign', 'Birt-hogg-dube syndrome', 'Birt-hogg-dube syndrome', 'Chickenpox', 'Chickenpox', 'Chickenpox', 'Cowden syndrome', 'Cowden syndrome', 'Dermatofibroma', 'Dermatofibroma', 'Disease by infectious agent', 'Disease by infectious agent', 'Eczema', 'Eczema', 'Exanthem', 'Exanthem', 'Exanthem', 'Herpes', 'Herpes', 'Herpes', 'Herpes', 'Lichen planus', 'Lichen planus', 'Lichen planus', 'Lichen planus', 'Lipoid proteinosis', 'Lipoid proteinosis', 'Lipoid proteinosis', 'Malign', 'Malign', 'Mali

process_data: 100%|██████████| 111/111 [00:48<00:00,  2.27it/s]

Overall accuracy@5: 110/111 = 99.099%

Per-class Accuracy@5:
Acanthosis nigricans (   3 samples): 100.000%
Acne                 (   2 samples): 100.000%
Acrodermatitis enteropathica (   2 samples): 100.000%
Actinic keratosis    (   2 samples): 100.000%
Addison's disease    (   4 samples): 100.000%
Amyloidosis          (   2 samples): 100.000%
Atopic dermatitis    (   4 samples): 100.000%
Basal Cell Carcinoma (   4 samples): 100.000%
Benign               (   2 samples): 100.000%
Birt-hogg-dube syndrome (   2 samples): 100.000%
Chickenpox           (   3 samples): 100.000%
Cowden syndrome      (   2 samples): 100.000%
Dermatofibroma       (   2 samples): 100.000%
Disease by infectious agent (   2 samples): 100.000%
Eczema               (   2 samples): 100.000%
Exanthem             (   3 samples): 100.000%
Herpes               (   4 samples): 100.000%
Lichen planus        (   4 samples): 100.000%
Malign               (   4 samples): 100.000%
Measles              (   3 samples): 100.000%
M

In [23]:
# single out of sample + symptomp use_replace = True
true_label = []
preds_top5 = []
val_dir = './out_of_sample'
random.seed(1234)
list_image = []
list_disease = []
list_symptom = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    all_images = [img for img in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, img))]
    
    num_rounds = random.randint(2, 4)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        num_image_samples = 1
        num_symptom_samples = 1

        sampled_images = rng.sample(all_images, num_image_samples)
        if sampled_images == []:
            continue
        if sampled_images in list_image:
            print("random trùng nhau", sampled_images)
        list_image.append([os.path.join(cls_dir, img_name) for img_name in sampled_images])
        list_disease.append(cls)


for disease in list_disease:

    key = f"{disease}".encode()
    seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
    rng = random.Random(seed)

    if disease == "Not a disease":
        list_symptom.append([])
    with open("test_symptom.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            if obj['original_name'] != disease:
                continue
            sampled_symptom = rng.sample(obj['used_symptoms'], num_symptom_samples)
            list_symptom.append(sampled_symptom)
            break
print(len(list_disease), list_disease)
print(len(list_image), list_image)
print(len(list_symptom), list_symptom)


true_label = []
preds_top5 = []

for image, symptom, disease in tqdm(zip(list_image, list_symptom, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_picture_symptom(list_picture=image, list_symptom=symptom, num_data=1000, top_k=5, use_replace = True, metric="cosine")
    names5 = [name for name, _, _, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

111 ['Acanthosis nigricans', 'Acanthosis nigricans', 'Acanthosis nigricans', 'Acne', 'Acne', 'Acrodermatitis enteropathica', 'Acrodermatitis enteropathica', 'Actinic keratosis', 'Actinic keratosis', "Addison's disease", "Addison's disease", "Addison's disease", "Addison's disease", 'Amyloidosis', 'Amyloidosis', 'Atopic dermatitis', 'Atopic dermatitis', 'Atopic dermatitis', 'Atopic dermatitis', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Benign', 'Benign', 'Birt-hogg-dube syndrome', 'Birt-hogg-dube syndrome', 'Chickenpox', 'Chickenpox', 'Chickenpox', 'Cowden syndrome', 'Cowden syndrome', 'Dermatofibroma', 'Dermatofibroma', 'Disease by infectious agent', 'Disease by infectious agent', 'Eczema', 'Eczema', 'Exanthem', 'Exanthem', 'Exanthem', 'Herpes', 'Herpes', 'Herpes', 'Herpes', 'Lichen planus', 'Lichen planus', 'Lichen planus', 'Lichen planus', 'Lipoid proteinosis', 'Lipoid proteinosis', 'Lipoid proteinosis', 'Malign', 'Malign', 'Mali

process_data: 100%|██████████| 111/111 [00:48<00:00,  2.27it/s]

Overall accuracy@5: 110/111 = 99.099%

Per-class Accuracy@5:
Acanthosis nigricans (   3 samples): 100.000%
Acne                 (   2 samples): 100.000%
Acrodermatitis enteropathica (   2 samples): 100.000%
Actinic keratosis    (   2 samples): 100.000%
Addison's disease    (   4 samples): 100.000%
Amyloidosis          (   2 samples): 100.000%
Atopic dermatitis    (   4 samples): 100.000%
Basal Cell Carcinoma (   4 samples): 100.000%
Benign               (   2 samples): 100.000%
Birt-hogg-dube syndrome (   2 samples): 100.000%
Chickenpox           (   3 samples): 100.000%
Cowden syndrome      (   2 samples): 100.000%
Dermatofibroma       (   2 samples): 100.000%
Disease by infectious agent (   2 samples): 100.000%
Eczema               (   2 samples): 100.000%
Exanthem             (   3 samples): 100.000%
Herpes               (   4 samples): 100.000%
Lichen planus        (   4 samples): 100.000%
Malign               (   4 samples): 100.000%
Measles              (   3 samples): 100.000%
M

In [28]:
# multi out of sample + symptomp use_replace = False

val_dir = './out_of_sample'
random.seed(1234)
list_image = []
list_disease = []
list_symptom = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    all_images = [img for img in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, img))]
    
    num_rounds = random.randint(2, 4)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        num_image_samples = rng.randint(2, 4)

        sampled_images = rng.sample(all_images, min(num_image_samples, len(all_images)))
        if sampled_images == []:
            continue
        if sampled_images in list_image:
            print("random trùng nhau", sampled_images)
        list_image.append([os.path.join(cls_dir, img_name) for img_name in sampled_images])
        list_disease.append(cls)


for disease in list_disease:
    

    key = f"{disease}".encode()
    seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
    rng = random.Random(seed)

    num_symptom_samples = rng.randint(2, 4)

    if disease == "Not a disease":
        list_symptom.append([])
    with open("test_symptom.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            if obj['original_name'] != disease:
                continue
            sampled_symptom = rng.sample(obj['used_symptoms'], min(num_symptom_samples, len(obj['used_symptoms'])))
            list_symptom.append(sampled_symptom)
            break
print(len(list_disease), list_disease)
print(len(list_image), list_image)
print(len(list_symptom), list_symptom)


true_label = []
preds_top5 = []

for image, symptom, disease in tqdm(zip(list_image, list_symptom, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_picture_symptom(list_picture=image, list_symptom=symptom, num_data=10000, top_k=5, use_replace = False, metric="cosine")
    names5 = [name for name, _, _, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

111 ['Acanthosis nigricans', 'Acanthosis nigricans', 'Acanthosis nigricans', 'Acne', 'Acne', 'Acrodermatitis enteropathica', 'Acrodermatitis enteropathica', 'Actinic keratosis', 'Actinic keratosis', "Addison's disease", "Addison's disease", "Addison's disease", "Addison's disease", 'Amyloidosis', 'Amyloidosis', 'Atopic dermatitis', 'Atopic dermatitis', 'Atopic dermatitis', 'Atopic dermatitis', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Benign', 'Benign', 'Birt-hogg-dube syndrome', 'Birt-hogg-dube syndrome', 'Chickenpox', 'Chickenpox', 'Chickenpox', 'Cowden syndrome', 'Cowden syndrome', 'Dermatofibroma', 'Dermatofibroma', 'Disease by infectious agent', 'Disease by infectious agent', 'Eczema', 'Eczema', 'Exanthem', 'Exanthem', 'Exanthem', 'Herpes', 'Herpes', 'Herpes', 'Herpes', 'Lichen planus', 'Lichen planus', 'Lichen planus', 'Lichen planus', 'Lipoid proteinosis', 'Lipoid proteinosis', 'Lipoid proteinosis', 'Malign', 'Malign', 'Mali

process_data: 100%|██████████| 111/111 [04:39<00:00,  2.52s/it]

Overall accuracy@5: 111/111 = 100.000%

Per-class Accuracy@5:
Acanthosis nigricans (   3 samples): 100.000%
Acne                 (   2 samples): 100.000%
Acrodermatitis enteropathica (   2 samples): 100.000%
Actinic keratosis    (   2 samples): 100.000%
Addison's disease    (   4 samples): 100.000%
Amyloidosis          (   2 samples): 100.000%
Atopic dermatitis    (   4 samples): 100.000%
Basal Cell Carcinoma (   4 samples): 100.000%
Benign               (   2 samples): 100.000%
Birt-hogg-dube syndrome (   2 samples): 100.000%
Chickenpox           (   3 samples): 100.000%
Cowden syndrome      (   2 samples): 100.000%
Dermatofibroma       (   2 samples): 100.000%
Disease by infectious agent (   2 samples): 100.000%
Eczema               (   2 samples): 100.000%
Exanthem             (   3 samples): 100.000%
Herpes               (   4 samples): 100.000%
Lichen planus        (   4 samples): 100.000%
Lipoid proteinosis   (   3 samples): 100.000%
Malign               (   4 samples): 100.000%


In [29]:
# multi out of sample + symptomp use_replace = True

val_dir = './out_of_sample'
random.seed(1234)
list_image = []
list_disease = []
list_symptom = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    all_images = [img for img in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, img))]
    
    num_rounds = random.randint(2, 4)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        num_image_samples = rng.randint(2, 4)

        sampled_images = rng.sample(all_images, min(num_image_samples, len(all_images)))
        if sampled_images == []:
            continue
        if sampled_images in list_image:
            print("random trùng nhau", sampled_images)
        list_image.append([os.path.join(cls_dir, img_name) for img_name in sampled_images])
        list_disease.append(cls)


for disease in list_disease:
    

    key = f"{disease}".encode()
    seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
    rng = random.Random(seed)

    num_symptom_samples = rng.randint(2, 4)

    if disease == "Not a disease":
        list_symptom.append([])
    with open("test_symptom.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            if obj['original_name'] != disease:
                continue
            sampled_symptom = rng.sample(obj['used_symptoms'], min(num_symptom_samples, len(obj['used_symptoms'])))
            list_symptom.append(sampled_symptom)
            break
print(len(list_disease), list_disease)
print(len(list_image), list_image)
print(len(list_symptom), list_symptom)


true_label = []
preds_top5 = []

for image, symptom, disease in tqdm(zip(list_image, list_symptom, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_picture_symptom(list_picture=image, list_symptom=symptom, num_data=10000, top_k=5, use_replace = True, metric="cosine")
    names5 = [name for name, _, _, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")

111 ['Acanthosis nigricans', 'Acanthosis nigricans', 'Acanthosis nigricans', 'Acne', 'Acne', 'Acrodermatitis enteropathica', 'Acrodermatitis enteropathica', 'Actinic keratosis', 'Actinic keratosis', "Addison's disease", "Addison's disease", "Addison's disease", "Addison's disease", 'Amyloidosis', 'Amyloidosis', 'Atopic dermatitis', 'Atopic dermatitis', 'Atopic dermatitis', 'Atopic dermatitis', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Basal Cell Carcinoma', 'Benign', 'Benign', 'Birt-hogg-dube syndrome', 'Birt-hogg-dube syndrome', 'Chickenpox', 'Chickenpox', 'Chickenpox', 'Cowden syndrome', 'Cowden syndrome', 'Dermatofibroma', 'Dermatofibroma', 'Disease by infectious agent', 'Disease by infectious agent', 'Eczema', 'Eczema', 'Exanthem', 'Exanthem', 'Exanthem', 'Herpes', 'Herpes', 'Herpes', 'Herpes', 'Lichen planus', 'Lichen planus', 'Lichen planus', 'Lichen planus', 'Lipoid proteinosis', 'Lipoid proteinosis', 'Lipoid proteinosis', 'Malign', 'Malign', 'Mali

process_data: 100%|██████████| 111/111 [03:43<00:00,  2.01s/it]

Overall accuracy@5: 111/111 = 100.000%

Per-class Accuracy@5:
Acanthosis nigricans (   3 samples): 100.000%
Acne                 (   2 samples): 100.000%
Acrodermatitis enteropathica (   2 samples): 100.000%
Actinic keratosis    (   2 samples): 100.000%
Addison's disease    (   4 samples): 100.000%
Amyloidosis          (   2 samples): 100.000%
Atopic dermatitis    (   4 samples): 100.000%
Basal Cell Carcinoma (   4 samples): 100.000%
Benign               (   2 samples): 100.000%
Birt-hogg-dube syndrome (   2 samples): 100.000%
Chickenpox           (   3 samples): 100.000%
Cowden syndrome      (   2 samples): 100.000%
Dermatofibroma       (   2 samples): 100.000%
Disease by infectious agent (   2 samples): 100.000%
Eczema               (   2 samples): 100.000%
Exanthem             (   3 samples): 100.000%
Herpes               (   4 samples): 100.000%
Lichen planus        (   4 samples): 100.000%
Lipoid proteinosis   (   3 samples): 100.000%
Malign               (   4 samples): 100.000%


In [31]:
# multi with false out of sample + symptom use_replace = False

val_dir = './out_of_sample'
random.seed(1234)
list_image = []
list_disease = []

all_classes = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    if os.path.isdir(cls_dir):
        imgs = [f for f in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, f))]
        all_classes.append((cls, cls_dir, imgs))

for cls, cls_dir, images in all_classes:

    num_rounds = max(len(images) // 2, 1)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        total_N = rng.randint(3, 5)

        other_choices = [
            (o_cls, o_dir, o_imgs)
            for (o_cls, o_dir, o_imgs) in all_classes
            if o_cls != cls and len(o_imgs) > 0
        ]
        if not other_choices:
            sampled_main = rng.sample(images, min(total_N, len(images)))
            batch = sampled_main
        else:
            o_cls, o_dir, o_imgs = rng.choice(other_choices)
            one_other = rng.choice(o_imgs)
            k_main = min(total_N - 1, len(images))
            sampled_main = rng.sample(images, k_main)
            batch = [os.path.join(o_dir, one_other)] + [os.path.join(cls_dir, img) for img in sampled_main]

        list_image.append(batch)
        list_disease.append(cls)

print(len(list_image), list_image)
print(len(list_disease), list_disease)

list_symptom = []
all_classes = []
with open("test_symptom.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        if obj['original_name'] in list_disease:
            all_classes.append((obj['original_name'], obj['used_symptoms']))

all_classes.append(("Not a disease", []))

symptom_dict = {d: syms for d, syms in all_classes}
for disease in list_disease:

    key = f"{disease}".encode()
    seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
    rng = random.Random(seed)

    total_N = rng.randint(2,4)
    symptoms = symptom_dict[disease]
    other_choices = [
        (o_disease, o_symptoms)
        for (o_disease, o_symptoms) in all_classes
        if o_disease != disease and len(o_symptoms) > 0
    ]
    if not other_choices:
        sampled_main = rng.sample(symptoms, min(total_N, len(symptoms)))
        batch = sampled_main
    else:
        o_disease, o_symptoms = rng.choice(other_choices)
        one_other = rng.choice(o_symptoms)
        k_main = min(total_N - 1, len(symptoms))
        sampled_main = rng.sample(symptoms, k_main)
        batch = [one_other] + sampled_main
    
    list_symptom.append(batch)
print(len(list_symptom), list_symptom)

true_label = []
preds_top5 = []

for image, symptom, disease in tqdm(zip(list_image, list_symptom, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_picture_symptom(list_picture=image, list_symptom = symptom, num_data = 10000, top_k = 5, use_replace = False, metric="cosine")
    names5 = [name for name, _, _, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")



83 [['./out_of_sample\\Lipoid proteinosis\\article_1487_cover_en_US.jpg', './out_of_sample\\Acanthosis nigricans\\S_0318_Acanthosis_nigricans_C0373908.origina.max-600x600.jpg', './out_of_sample\\Acanthosis nigricans\\Treatment-for-Acanthosis-Nigricans.png', './out_of_sample\\Acanthosis nigricans\\Acan_nig_axil_2615addf7111d83624b7a7a5f3466fb3.jpg', './out_of_sample\\Acanthosis nigricans\\A_0917_acanthosis-nigricans_C5HE23.original.max-600x600.jpg'], ['./out_of_sample\\Cowden syndrome\\images (19).jpg', './out_of_sample\\Acanthosis nigricans\\ch1509.fig1_.jpg', './out_of_sample\\Acanthosis nigricans\\S_0318_Acanthosis_nigricans_C0373908.origina.max-600x600.jpg'], ['./out_of_sample\\Melanocytic Nevi\\reed-naevus.jpg', './out_of_sample\\Acne\\2356.jpg', './out_of_sample\\Acne\\acne-face-2-18.jpg'], ['./out_of_sample\\Malign\\C0044260-Melanoma_skin_cancer-SPL.original.max-600x600.jpg', './out_of_sample\\Acne\\unnamed.jpg', './out_of_sample\\Acne\\acne-face-2-18.jpg', './out_of_sample\\Acne

process_data: 100%|██████████| 83/83 [05:24<00:00,  3.90s/it]

Overall accuracy@5: 80/83 = 96.386%

Per-class Accuracy@5:
Acanthosis nigricans (   2 samples): 100.000%
Acne                 (   2 samples): 100.000%
Acrodermatitis enteropathica (   2 samples): 100.000%
Actinic keratosis    (   2 samples): 100.000%
Addison's disease    (   2 samples): 100.000%
Amyloidosis          (   2 samples): 100.000%
Atopic dermatitis    (   2 samples): 100.000%
Basal Cell Carcinoma (   2 samples): 100.000%
Benign               (   2 samples): 100.000%
Birt-hogg-dube syndrome (   2 samples): 100.000%
Chickenpox           (   2 samples): 100.000%
Cowden syndrome      (   3 samples): 100.000%
Dermatofibroma       (   3 samples): 100.000%
Disease by infectious agent (   2 samples): 100.000%
Eczema               (   2 samples): 100.000%
Exanthem             (   2 samples): 100.000%
Herpes               (   2 samples): 100.000%
Lichen planus        (   2 samples): 100.000%
Malign               (   2 samples): 100.000%
Measles              (   2 samples): 100.000%
Mel

In [32]:
# multi with false out of sample + symptom use_replace = True

val_dir = './out_of_sample'
random.seed(1234)
list_image = []
list_disease = []

all_classes = []
for cls in os.listdir(val_dir):
    cls_dir = os.path.join(val_dir, cls)
    if os.path.isdir(cls_dir):
        imgs = [f for f in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, f))]
        all_classes.append((cls, cls_dir, imgs))

for cls, cls_dir, images in all_classes:

    num_rounds = max(len(images) // 2, 1)
    for i in range(num_rounds):

        key = f"{cls}-{i}".encode()
        seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
        rng = random.Random(seed)

        total_N = rng.randint(3, 5)

        other_choices = [
            (o_cls, o_dir, o_imgs)
            for (o_cls, o_dir, o_imgs) in all_classes
            if o_cls != cls and len(o_imgs) > 0
        ]
        if not other_choices:
            sampled_main = rng.sample(images, min(total_N, len(images)))
            batch = sampled_main
        else:
            o_cls, o_dir, o_imgs = rng.choice(other_choices)
            one_other = rng.choice(o_imgs)
            k_main = min(total_N - 1, len(images))
            sampled_main = rng.sample(images, k_main)
            batch = [os.path.join(o_dir, one_other)] + [os.path.join(cls_dir, img) for img in sampled_main]

        list_image.append(batch)
        list_disease.append(cls)

print(len(list_image), list_image)
print(len(list_disease), list_disease)

list_symptom = []
all_classes = []
with open("test_symptom.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        if obj['original_name'] in list_disease:
            all_classes.append((obj['original_name'], obj['used_symptoms']))

all_classes.append(("Not a disease", []))

symptom_dict = {d: syms for d, syms in all_classes}
for disease in list_disease:

    key = f"{disease}".encode()
    seed = int(hashlib.md5(key).hexdigest(), 16) & 0xFFFFFFFF
    rng = random.Random(seed)

    total_N = rng.randint(2,4)
    symptoms = symptom_dict[disease]
    other_choices = [
        (o_disease, o_symptoms)
        for (o_disease, o_symptoms) in all_classes
        if o_disease != disease and len(o_symptoms) > 0
    ]
    if not other_choices:
        sampled_main = rng.sample(symptoms, min(total_N, len(symptoms)))
        batch = sampled_main
    else:
        o_disease, o_symptoms = rng.choice(other_choices)
        one_other = rng.choice(o_symptoms)
        k_main = min(total_N - 1, len(symptoms))
        sampled_main = rng.sample(symptoms, k_main)
        batch = [one_other] + sampled_main
    
    list_symptom.append(batch)
print(len(list_symptom), list_symptom)

true_label = []
preds_top5 = []

for image, symptom, disease in tqdm(zip(list_image, list_symptom, list_disease), total=len(list_disease), desc="process_data"):

    top5 = my_model.find_query_by_picture_symptom(list_picture=image, list_symptom = symptom, num_data = 10000, top_k = 5, use_replace = True, metric="cosine")
    names5 = [name for name, _, _, _ in top5]

    true_label.append(disease)
    preds_top5.append(names5)


total = len(true_label)
correct_overall = sum(1 for true, top5 in zip(true_label, preds_top5)
                      if true in top5)
recall5_overall = correct_overall / total * 100
print(f"Overall accuracy@5: {correct_overall}/{total} = {recall5_overall:.3f}%")


counts = defaultdict(int)
corrects = defaultdict(int)

for true, top5 in zip(true_label, preds_top5):
    counts[true] += 1
    if true in top5:
        corrects[true] += 1

acc_list = []
for cls in counts:
    recall5 = corrects[cls] / counts[cls] * 100
    acc_list.append((cls, counts[cls], recall5))

acc_list.sort(key=lambda x: x[2], reverse=True)

print("\nPer-class Accuracy@5:")
for cls, cnt, rec in acc_list:
    print(f"{cls:<20} ({cnt:>4} samples): {rec:6.3f}%")



83 [['./out_of_sample\\Lipoid proteinosis\\article_1487_cover_en_US.jpg', './out_of_sample\\Acanthosis nigricans\\S_0318_Acanthosis_nigricans_C0373908.origina.max-600x600.jpg', './out_of_sample\\Acanthosis nigricans\\Treatment-for-Acanthosis-Nigricans.png', './out_of_sample\\Acanthosis nigricans\\Acan_nig_axil_2615addf7111d83624b7a7a5f3466fb3.jpg', './out_of_sample\\Acanthosis nigricans\\A_0917_acanthosis-nigricans_C5HE23.original.max-600x600.jpg'], ['./out_of_sample\\Cowden syndrome\\images (19).jpg', './out_of_sample\\Acanthosis nigricans\\ch1509.fig1_.jpg', './out_of_sample\\Acanthosis nigricans\\S_0318_Acanthosis_nigricans_C0373908.origina.max-600x600.jpg'], ['./out_of_sample\\Melanocytic Nevi\\reed-naevus.jpg', './out_of_sample\\Acne\\2356.jpg', './out_of_sample\\Acne\\acne-face-2-18.jpg'], ['./out_of_sample\\Malign\\C0044260-Melanoma_skin_cancer-SPL.original.max-600x600.jpg', './out_of_sample\\Acne\\unnamed.jpg', './out_of_sample\\Acne\\acne-face-2-18.jpg', './out_of_sample\\Acne

process_data: 100%|██████████| 83/83 [05:32<00:00,  4.01s/it]

Overall accuracy@5: 80/83 = 96.386%

Per-class Accuracy@5:
Acanthosis nigricans (   2 samples): 100.000%
Acne                 (   2 samples): 100.000%
Acrodermatitis enteropathica (   2 samples): 100.000%
Actinic keratosis    (   2 samples): 100.000%
Addison's disease    (   2 samples): 100.000%
Amyloidosis          (   2 samples): 100.000%
Atopic dermatitis    (   2 samples): 100.000%
Basal Cell Carcinoma (   2 samples): 100.000%
Benign               (   2 samples): 100.000%
Birt-hogg-dube syndrome (   2 samples): 100.000%
Chickenpox           (   2 samples): 100.000%
Cowden syndrome      (   3 samples): 100.000%
Dermatofibroma       (   3 samples): 100.000%
Disease by infectious agent (   2 samples): 100.000%
Eczema               (   2 samples): 100.000%
Exanthem             (   2 samples): 100.000%
Herpes               (   2 samples): 100.000%
Lichen planus        (   2 samples): 100.000%
Malign               (   2 samples): 100.000%
Measles              (   2 samples): 100.000%
Mel